In [2]:
!git clone https://github.com/ultralytics/yolov5.git
%cd yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 16927, done.
remote: Counting objects: 100% (122/122), done.
remote: Compressing objects: 100% (75/75), done.
remote: Total 16927 (delta 62), reused 86 (delta 47), pack-reused 16805 (from 1)
Receiving objects: 100% (16927/16927), 15.69 MiB | 22.44 MiB/s, done.
Resolving deltas: 100% (11595/11595), done.
/home/kestrel41x4/Desktop/CodeProjects/asl_detection/final_data/yolov5


In [4]:
# install dependencies as necessary
!pip install -qr requirements.txt  # install dependencies (ignore errors)
import torch

from IPython.display import Image, clear_output  # to display images
# from utils.google_utils import gdrive_download  # to download models/datasets

# clear_output()
print('Setup complete. Using torch %s %s' % (torch.__version__, torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'))

Setup complete. Using torch 2.4.0+cu121 CPU


In [10]:
import yaml
with open("/home/kestrel41x4/Desktop/CodeProjects/asl_detection/final_data/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [12]:
num_classes

'6'

In [1]:
%cat models/yolov5s.yaml



cat: models/yolov5s.yaml: No such file or directory


In [62]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))